# Entendimiento de Datos
En este cuaderno revisaremos dos aspectos grandes del entendimiento de datos: el perfilamiento y la calidad de los datos. Adicionalmente, se dan una serie de funciones para facilitar la manipulación de los datos.
* Inicio
    * Carga
    * Manipulación Básica

* Perfilamiento de Datos
    * Exploración
    * Visualización

* Calidad de Datos:
  * Completitud
  * Duplicados
  * Estandarización

* Resumenes automáticos para el entendimiento

Para la limpieza de los datos utilizaremos la libreria de **Pandas** (https://pandas.pydata.org/) y para la visualización de los datos, usaremos: **Seaborn**(https://seaborn.pydata.org/) y **Matplotlib** (https://matplotlib.org/).

## Los Datos
Trabajaremos con una base de datos de accidentes de BiciAlpes.

La base de datos original, la pueden encontrar aquí: **

# 1. Inicio

## 1.1 Carga

### 1.1.1 *Limpieza y lemantización*

In [20]:
# Librerias generales
# Pandas
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Número máximo de filas a mostar
# Ranom seed
import numpy as np
np.random.seed(3301)

# Seaborn
import seaborn as sns

# Matplolib
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px

#Limpieza de datos

!pip install spacy
!python -m spacy download es_core_news_sm

import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import re
import unicodedata
import pandas as pd

# Descargar e inicializar spaCy en español
import os
if not os.path.exists(spacy.util.get_package_path("es_core_news_sm")):
    spacy.cli.download("es_core_news_sm")

nlp = spacy.load("es_core_news_sm", disable=["ner", "textcat"])



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 64.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [21]:
# Ubicación de la base de datos
db_location = '/content/Fake News Español Proyecto 1.csv'

In [22]:
# Cargar los datos
df = pd.read_csv(db_location, sep=';', encoding = "utf-8")

In [23]:
# Dimensiones de los datos
df.shape

(57063, 5)

In [24]:
# Ver los datos
display(df.sample(5)) # Muestra
#display(df_bicis.head(5)) # Primeras Filas
#display(df_bicis.tail(5)) # Ultimas Filas

,ID,Label,Titulo,Descripcion,Fecha
40153,ID,0,Torra se querella contra Martina Velarde por a...,Torra y compañía dicen que tenemos un gen meno...,04/12/2018
38869,ID,1,Esperanza Aguirre niega que ordenara espiar a ...,La Audiencia Provincial de Madrid ha reanudado...,11/02/2019
52222,ID,0,Tres mujeres dan la sorpresa en la jornada ele...,"Cuca Gamarra (UPN), Maddalen Iriarte (Izquierd...",13/07/2020
8542,ID,1,"ERC, molesto con el PSOE por dar por hecho ya ...",Sánchez aseguró que el pacto va a estar siempr...,05/12/2019
9977,ID,1,"Del 'Pablo, tenemos 12 días para ganar' de Gab...",Repasamos algunos de los momentos más sonados ...,22/04/2021


Eliminación de la columna ID porque todos las filas lo tenián con el valor 'ID'

In [25]:
df1 = df.drop("ID", axis = 1)

Revisamos cuantos valores nulos hay por columna

In [26]:
valores_nulos = df1.isnull().sum()
print(valores_nulos)

Label           0
Titulo         16
Descripcion     0
Fecha           0
dtype: int64


Como en titulo hay 16 valores nulos, que es una cantida mínima comparado con la cantidad de elementos en el dataset. Los eliminamos

In [27]:
df2 = df1.dropna()

Vamos ahora a revisar si hay elementos duplicados en la columna titulo

In [28]:
duplicados_titulo = df2['Titulo'].duplicated().sum()
print(f"Duplicados según titulo: {duplicados_titulo}")

Duplicados según titulo: 5443


In [29]:
valores_duplicados = df2[df2.duplicated(keep=False)]
valores_duplicados_ordenados = valores_duplicados.sort_values(by=df2.columns.tolist())
print(valores_duplicados_ordenados)

       Label                                             Titulo  \
24343      0  El Nueva Canarias cierra filas con Rita Maestr...   
29842      0  El Nueva Canarias cierra filas con Rita Maestr...   
1743       0  Escándalo de corrupción salpica a líderes sind...   
2981       0  Escándalo de corrupción salpica a líderes sind...   
18253      0  Escándalo de corrupción salpica a líderes sind...   
...      ...                                                ...   
9206       1  ¿Por qué las energías verdes están causando un...   
28272      1  ¿Qué son las políticas activas de empleo y por...   
37769      1  ¿Qué son las políticas activas de empleo y por...   
29781      1  Àngels Barceló: 'Los demócratas somos muchos más'   
35301      1  Àngels Barceló: 'Los demócratas somos muchos más'   

                                             Descripcion       Fecha  
24343  El Comité Federal termina con una votación uná...  10/06/2023  
29842  El Comité Federal termina con una votación uná

Después de analizar las filas repetidas y corroborar que el "label" es el mismo y que no se trataba de la misma noticia pero con una descripción diferente o algún diferenciador. Procedemos a eliminarlas

In [30]:
df3 = df2.drop_duplicates(subset = ['Titulo'])

In [31]:
duplicados_titulo = df3['Titulo'].duplicated().sum()
print(f"Duplicados según titulo: {duplicados_titulo}")

Duplicados según titulo: 0


Ahora vamos procesar y limpíar el texto, para poder tener una mejor deteción de patrones

In [34]:
import nltk
nltk.download('stopwords')

def limpiar_texto(texto):
    texto = texto.lower()
    tokens = word_tokenize(texto, language='spanish')

    stop_words = set(stopwords.words('spanish'))
    palabras_filtradas = [palabra for palabra in tokens if palabra not in stop_words]

    texto_limpio = ' '.join(palabras_filtradas)
    return texto_limpio


df3['Titulo'] = df3["Titulo"].apply(limpiar_texto)
df3['Descripcion'] = df3["Descripcion"].apply(limpiar_texto)

display(df3.head(5)) # Primeras Filas


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-34-6a69f43e9a86>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Titulo'] = df3["Titulo"].apply(limpiar_texto)
<ipython-input-34-6a69f43e9a86>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Descripcion'] = df3["Descripcion"].apply(limpiar_texto)


,Label,Titulo,Descripcion,Fecha
0,1,'the guardian ' va sánchez : 'europa necesita ...,diario británico publicó pasado jueves editori...,02/06/2023
1,0,revelan gobierno negocio liberación mireles ca...,revelan gobierno negocio liberación mireles ca...,01/10/2023
2,1,'ahora nunca ' joan fuster estatuto valenciano...,valencianismo convoca castelló fiesta grande c...,25/04/2022
3,1,"iglesias alienta yolanda díaz , erc eh bildu n...","política , igual negociar empresarios , negoci...",03/01/2022
4,0,puigdemont : 'no ninguna tragedia repetición e...,"entrevista punt avui , líder jxcat desdramatiz...",09/03/2018


In [35]:
def lematizar_sin_tildes(texts):
    docs = nlp.pipe(texts, batch_size=500)
    textos_lematizados = [" ".join([token.lemma_ for token in doc if not token.is_punct])
                         for doc in docs]
    return [unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
            for texto in textos_lematizados]

df4 = df3.copy()
df4['Titulo'] = lematizar_sin_tildes(df4['Titulo'])
df4['Descripcion'] = lematizar_sin_tildes(df4['Descripcion'])

In [36]:
display(df4.head(5)) # Primeras Filas

,Label,Titulo,Descripcion,Fecha
0,1,the guardiar ir sanchez europa necesitar apues...,diario britanico publicar pasado jueves editor...,02/06/2023
1,0,revelar gobierno negocio liberacion mirel camb...,revelar gobierno negocio liberacion mirel camb...,01/10/2023
2,1,ahora nunca joan fuster estatuto valenciano cu...,valencianismo convocar castellar fiesta grande...,25/04/2022
3,1,iglesia alentar yolanda diaz erc eh bildu nego...,politico igual negociar empresario negociar gr...,03/01/2022
4,0,puigdemont no ninguno tragedia repeticion elec...,entrevista punt avui lider jxcat desdramatizad...,09/03/2018


## 1.2 Manipulación Básica

En esta parte del Cuaderno la idea es que se familarice con algunos comandos que van a permitir manipular mejor los datos y avanzar en la comprensión de los mismos, muy de la mano del diccionario de datos.

In [37]:
df4.dtypes

,0
Label,int64
Titulo,object
Descripcion,object
Fecha,object


In [38]:
# Resumen de las principales estadísticas de las variables numéricas
df4['Label'].describe()

,Label
count,51604.000000
mean,0.587571
std,0.492276
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


#### 1.2.1 Datos de Fechas

In [39]:
# la columna Fecha deberia ser fecha pero es object
df4.Fecha.tail(10)

,Fecha
57050,17/01/2020
57051,19/01/2019
57053,07/05/2018
57055,12/03/2023
57056,21/10/2018
57057,23/12/2021
57058,08/06/2021
57059,08/09/2020
57060,12/07/2018
57061,13/02/2022


In [40]:
df5 = df4.copy()

df5['Fecha'] = pd.to_datetime(df5.Fecha, dayfirst= True, errors = 'coerce')
df5['Fecha'].tail(10)

,Fecha
57050,2020-01-17
57051,2019-01-19
57053,2018-05-07
57055,2023-03-12
57056,2018-10-21
57057,2021-12-23
57058,2021-06-08
57059,2020-09-08
57060,2018-07-12
57061,2022-02-13


In [41]:
print(df5["Fecha"].isna().sum())

0


# 2. Vectorización del texto

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b[a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]{2,}\b", max_features=5000)
X_tfidf = vectorizer.fit_transform(df["Titulo"] + " " + df["Descripcion"])

# Convertir a DataFrame
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
print(tfidf_df.head())

,00,000,0000,004,0083,00h,01,016,017,02,021,024,...,zuarth,zubia,zubiaga,zubieta,zubio,zuera,zufre,zulo,zuloago,zulueta,zurekin,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 3. Visualización de los datos

### 2.4.1 Diagramas de Temporales

Distribución de las noticias según si son verdaderas o falsas

In [ ]:
conteo_clases = df5["Label"].value_counts()

mapeo_etiquetas = {0: "Noticias Falsas", 1: "Noticias Verdaderas"}
fig = px.pie(names=conteo_clases.index.map(mapeo_etiquetas), values=conteo_clases.values,
             title=f'Distribución de noticias falsas y verdaderas ({df.shape[0]} noticias)')

fig.show()

Dado que la distribución entre noticias falsas y verdaderas es relativamente equilibrada, podemos trabajar con los datos sin necesidad de ajustar el balance de clases.

---
Analisis para identificar si la fecha esta directamente relacionada con la veracidad de una noticia

In [ ]:
print("Rango de fechas:", df5["Fecha"].min(), "a", df5["Fecha"].max())


# Distribución de noticias falsas vs. verdaderas por año
plt.figure(figsize=(10,5))
df5.groupby([df5["Fecha"].dt.year, "Label"]).size().unstack().rename(columns={0: "Falsas", 1: "Verdaderas"}).plot(kind="bar", stacked=True, figsize=(10,5))
plt.title("Distribución de noticias falsas y verdaderas por año")
plt.xlabel("Año")
plt.ylabel("Cantidad de noticias")
plt.legend(["Verdaderas (1)", "Falsas (0)"])
plt.show()

# Análisis de correlación entre año y etiqueta de noticia
correlacion = df5["Fecha"].dt.year.corr(df["Label"])
print("Correlación entre año y etiqueta de noticia:", correlacion)

## 4. Reportes Automáticos para realizar el entendimiento de los datos

Para los reportes automáticos, se puede usar al herramienta de pandas profiling.


Para cada columna, genera las siguientes estadísticas, si son relevantes para el tipo de columna, se presentan en un informe HTML interactivo:

1. Inferencia de tipo: detecta los tipos de columnas en un dataframe.
2. Esenciales: tipo, valores únicos, valores faltantes.
3. Estadísticas de cuantiles como valor mínimo, Q1, mediana, Q3, máximo, rango, rango intercuartílico. Esta opción es bastante útil para identificar datos atípicos.
4. Estadísticas descriptivas como media, moda, desviación estándar, suma, desviación absoluta mediana, coeficiente de variación, curtosis, asimetría.
5. Valores más frecuentes.
6. Histogramas.
7. Correlaciones destacando variables altamente correlacionadas, matrices de Spearman, Pearson y Kendall. Esto permite descubrir relaciones entre atributos.
8. Matriz de valores faltantes, recuento, mapa de calor y dendrograma de valores faltantes

Tomado de la librería oficial de pandas_profiling en [github](https://github.com/pandas-profiling/pandas-profiling)

Lo más importante al utilizar esta librería es recordar que lo fundamental son los análisis que hagamos sobre estos reportes.

In [ ]:
import pandas_profiling

profiling =pandas_profiling.ProfileReport(df5)
profiling

In [ ]:
profiling.to_file("Proyecto1_db_profile.html")